# Lab 1 Report

I implemented scalar and parallel mandelbrot algorithms on both the CPU and GPU. I've included the images they generate and their runtimes.

<table>
  <tr>
    <th></th>
    <th>GPU</th>
    <th>CPU</th>
  </tr>
  <tr>
    <th>Scalar</th>
    <td> 311.799 ms <br> <img src="telerun_gpu/mandelbrot_gpu_scalar.bmp" width="200"/></td>
    <td> 40.925 ms <br> <img src="telerun_cpu/mandelbrot_cpu_scalar.bmp" width="200"/></td>
  </tr>
  <tr>
    <th>Vectorized</th>
    <td>14.1404 ms <br> <img src="telerun_gpu/mandelbrot_gpu_vector.bmp" width="200"/></td>
    <td> 4.09366 ms <br> <img src="telerun_cpu/mandelbrot_cpu_vector.bmp" width="200"/></td>
  </tr>
</table>

# Question 1

*How does the run time of the scalar GPU implementation compare to the scalar CPU implementation? What factors do you think might contribute to each of their run times?*

The scalar CPU implementation runs in 40ms (telerun job 0190625) and the scalar GPU implementation runs in 311ms. GPUs are faster than CPUs for highly parallelize tasks, but here we do not parallelize anything which is why the GPU does not give a speedup.

*Commands:*

Running on the CPU `python telerun/telerun.py submit lab1/mandelbrot_cpu.cpp` outputs
```
    Testing with image size 256x256 and 1000 max iterations.
    Running mandelbrot_cpu_scalar ...
      Runtime: 40.925 ms
    Running mandelbrot_cpu_vector ...
      Runtime: 4.09366 ms
      Correctness: average output difference from reference = 0.000218033
```

Running on the GPU `python telerun/telerun.py submit lab1/mandelbrot_gpu.cu` outputs
```bash
    Testing with image size 256x256 and 1000 max iterations.
    Running launch_mandelbrot_gpu_scalar ...
      Runtime: 311.799 ms
      Correctness: average output difference from reference = 0
    Running launch_mandelbrot_gpu_vector ...
      Runtime: 14.1404 ms
      Correctness: average output difference from reference 0
```

# Question 2

*How did you initialize cx and cy? How did you handle differences in the number of inner loop iterations between pixels in the same vector? How does the performance of your vectorized implementation compare to the scalar versions?*

To parallelize the mandelbrot computation, I iterated over one 16-wide row of pixels at a time. This means the outer loop (row `i`) stays the same but the inner loop (column `j`) now processes 16 elements at a time and is wirtten as `for (uint64_t j = 0; j < img_size; j+=VECTOR_SIZE)`.

To compute `cx`, `cy`, I needed to access a vector of 16 indices `i`, `j` at once. Importantly, the vectorized row `i` contains 16 of the same values so that for all elements in the vector we are looking at the same row. But the vectorized column `j` contains 16 different values so that we can look at 16 different columns at the same time. To implement this, I did
```cpp
__m512 range = _mm512_set_ps(15.0, 14.0, 13.0, 12.0, 11.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 0.0);
i_vec = _mm512_set1_ps(i);
j_vec = _mm512_add_ps(_mm512_set1_ps(j), range);
```

The while loop is tricky because it can run for different number of iterations for different vectorized elements. To get around this, I used a mask that is True when `x2+y2<=4` and we only update `iters_vec` in the while loop for the elements where this mask is True. Here is how I computed the mask
```cpp
__mmask16 is_less_than_four(__m512 x2_vec, __m512 y2_vec, __mmask16 mask, __m512 zeros, __m512 fours) {
    __m512 x_plus_y = _mm512_mask_add_ps(zeros, mask, x2_vec, y2_vec);
    __mmask16 less_than_four = _mm512_mask_cmple_ps_mask(mask, x_plus_y, fours);
    return less_than_four;
}
```

The CPU vectorized implementation is 10x faster than the CPU scalar implementation. `python telerun/telerun.py submit lab1/mandelbrot_cpu.cpp` outputs
```
    Testing with image size 256x256 and 1000 max iterations.
    Running mandelbrot_cpu_scalar ...
      Runtime: 40.9261 ms
    Running mandelbrot_cpu_vector ...
      Runtime: 4.08268 ms
      Correctness: average output difference from reference = 0.000218033
```
However, ideally, our vectorized implementation should be 16x faster, not 10x faster. I believe there is some overhead of doing things in parallel that prevents us from getting the full 16x speedup. Also, we have to perform extra operations in the vectorized implementation, like initializing a mask, which we don't need to do in the scalar version. Additionally, I imagine branch prediction works very well for scalar programs but less so for vectorized programs, causing a slight slow-down.

# Question 3

*How does the performance of your vectorized implementation compare to the scalar versions? Given how you implemented the vector-parallel CPU version with explicit SIMD, how do you think the GPU executes multiple kernel instances that run different numbers of iterations?*

The GPU scalar implementation runs in 311 ms and the GPU vectorized implementation runs in 14 ms (telerun job 0190615). This is a 22x speedup, larger than the 10x speedup we saw for the CPU vectorized implementation. However, the GPU runs 32 thread in parallel so in an ideal world we get a 32x speedup but we only got a 22x speedup. Why? Here are some ideas:

1. The while loop will terminate earlier for some threads and later for other threads. Since we have to wait for all threads in the same warp to finish their computation, this means that many threads can sit idle while waiting for their other warp friends to finish. We are wasting time and not using our GPUs efficiently.
2. I'm sure there are some bad memory access patterns -- missing caches, memory bank conflicts -- since we didn't optimize for these. But this is a small factor.

How does the GPU execute multiple kernel instances that run for different number of iterations? This is a great question. We must wait for all the threads in the warp to finish computing in order to start computing new things. This issue is called *thread divergence*, when different threads within a warp take different execution paths through the code. Our GPU handles warp divergence by executing all paths *serially* on all warps and you can think of it as then applying a mask to all the warps where this code execution path is not relevant. In the Mandelbrot code, some threads exit after 1 iteration and others may exit after 10 iterations. The threads that exit early must wait for the slowest thread in the warp to finish. More technically, we are still running the code on all the threads in the warp but are able to, metaphorically, apply a mask so we only actually update the computation for the relevant threads in that code path.

`python telerun/telerun.py submit lab1/mandelbrot_gpu.cu` outputs
```bash
    Testing with image size 256x256 and 1000 max iterations.
    Running launch_mandelbrot_gpu_scalar ...
      Runtime: 311.799 ms
      Correctness: average output difference from reference = 0
    Running launch_mandelbrot_gpu_vector ...
      Runtime: 14.1404 ms
      Correctness: average output difference from reference 0
```